In [1]:
#Training for GAN

from __future__ import print_function
import os
## setting seed ###
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
import random
random.seed(1)
os.environ['PYTHONHASHSEED'] = '0' 
##################

from collections import defaultdict
try:
    import cPickle as pickle
except ImportError:
    import pickle
import keras
import argparse
import sys
import h5py 
import numpy as np
import time
import math
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import analysis.utils.GANutils as gan
if os.environ.get('HOSTNAME') == 'tlab-gpu-oldeeptector.cern.ch': # Here a check for host can be used to set defaults accordingly
    tlab = True
else:
    tlab= False
    
try:
    import setGPU #if Caltech
except:
    pass

from memory_profiler import profile # used for memory profiling
import keras.backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adadelta, Adam, RMSprop
from keras.utils.generic_utils import Progbar
config = tf.ConfigProto(log_device_placement=True)

Using TensorFlow backend.


# Functions

In [2]:
def get_parser():
    parser = argparse.ArgumentParser(description='3D GAN Params' )
    parser.add_argument('--nbepochs', action='store', type=int, default=60, help='Number of epochs to train for.')
    parser.add_argument('--batchsize', action='store', type=int, default=64, help='batch size per update')
    parser.add_argument('--latentsize', action='store', type=int, default=256, help='size of random N(0, 1) latent space to sample')
    parser.add_argument('--datapath', action='store', type=str, default='/eos/user/g/gkhattak/VarAngleData/*Measured3ThetaEscan/*.h5', help='HDF5 files to train from.')
    parser.add_argument('--dformat', action='store', type=str, default='channels_last')
    parser.add_argument('--nEvents', action='store', type=int, default=200000, help='Maximum Number of events used for Training')
    parser.add_argument('--verbose', action='store_true', help='Whether or not to use a progress bar')
    parser.add_argument('--xscale', action='store', type=int, default=1, help='Multiplication factor for ecal deposition')
    parser.add_argument('--xpower', action='store', type=float, default=0.85, help='pre processing of cell energies by raising to a power')
    parser.add_argument('--yscale', action='store', type=int, default=100, help='Division Factor for Primary Energy.')
    parser.add_argument('--ascale', action='store', type=int, default=1, help='Multiplication factor for angle input')
    parser.add_argument('--analyse', action='store', default=False, help='Whether or not to perform analysis')
    parser.add_argument('--gen_weight', action='store', type=float, default=3, help='loss weight for generation real/fake loss')
    parser.add_argument('--aux_weight', action='store', type=float, default=0.1, help='loss weight for auxilliary energy regression loss')
    parser.add_argument('--ang_weight', action='store', type=float, default=25, help='loss weight for angle loss')
    parser.add_argument('--ecal_weight', action='store', type=float, default=0.1, help='loss weight for ecal sum loss')
    parser.add_argument('--hist_weight', action='store', type=float, default=0.1, help='loss weight for additional bin count loss')
    parser.add_argument('--thresh', action='store', type=int, default=0., help='Threshold for cell energies')
    parser.add_argument('--angtype', action='store', type=str, default='mtheta', help='Angle to use for Training. It can be theta, mtheta or eta')
    parser.add_argument('--particle', action='store', type=str, default='Ele', help='Type of particle')
    parser.add_argument('--lr', action='store', type=float, default=0.0008, help='Learning rate')
    parser.add_argument('--name', action='store', type=str, default='training', help='Unique identifier can be set for each training')
    return parser

# A histogram fucntion that counts cells in different bins
def hist_count(x, p=1.0, daxis=(1, 2, 3)):
    limits=np.array([0.05, 0.03, 0.02, 0.0125, 0.008, 0.003]) # bin boundaries used
    limits= np.power(limits, p)
    bin1 = np.sum(np.where(x>(limits[0]) , 1, 0), axis=daxis)
    bin2 = np.sum(np.where((x<(limits[0])) & (x>(limits[1])), 1, 0), axis=daxis)
    bin3 = np.sum(np.where((x<(limits[1])) & (x>(limits[2])), 1, 0), axis=daxis)
    bin4 = np.sum(np.where((x<(limits[2])) & (x>(limits[3])), 1, 0), axis=daxis)
    bin5 = np.sum(np.where((x<(limits[3])) & (x>(limits[4])), 1, 0), axis=daxis)
    bin6 = np.sum(np.where((x<(limits[4])) & (x>(limits[5])), 1, 0), axis=daxis)
    bin7 = np.sum(np.where((x<(limits[5])) & (x>0.), 1, 0), axis=daxis)
    bin8 = np.sum(np.where(x==0, 1, 0), axis=daxis)
    bins = np.concatenate([bin1, bin2, bin3, bin4, bin5, bin6, bin7, bin8], axis=1)
    bins[np.where(bins==0)]=1 # so that an empty bin will be assigned a count of 1 to avoid unstability
    return bins

#get data for training
def GetDataAngle(datafile, xscale =1, xpower=1, yscale = 100, angscale=1, angtype='theta', thresh=1e-4, daxis=4):
    print ('Loading Data from .....', datafile)
    f=h5py.File(datafile,'r')
    ang = np.array(f.get(angtype))
    X=np.array(f.get('ECAL'))* xscale
    Y=np.array(f.get('energy'))/yscale
    X[X < thresh] = 0
    X = X.astype(np.float32)
    Y = Y.astype(np.float32)
    ang = ang.astype(np.float32)
    ecal = np.sum(X, axis=(1, 2, 3))
    X = np.expand_dims(X, axis=daxis)
    if xpower !=1.:
        X = np.power(X, xpower)
    return X, Y, ang, ecal

# Imports

In [3]:
from AngleArch3dGAN import generator, discriminator

In [4]:
parser = get_parser()
#params = parser.parse_args()
params = parser.parse_args("--datapath=../../3dgan_files/data/LCD/EleMeasuredThetaEscan/*.h5".split())
nb_epochs = params.nbepochs #Total Epochs
batch_size = params.batchsize #batch size
latent_size = params.latentsize #latent vector size
verbose = params.verbose
datapath = params.datapath# Data path
nEvents = params.nEvents# maximum number of events used in training
ascale = params.ascale # angle scale
yscale = params.yscale # scaling energy
weightdir = 'weights/3dgan_weights_' + params.name
pklfile = 'results/3dgan_history_' + params.name + '.pkl'# loss history
resultfile = 'results/3dgan_analysis' + params.name + '.pkl'# optimization metric history
xscale = params.xscale
xpower = params.xpower
analyse=params.analyse # if analysing
loss_weights=[params.gen_weight, params.aux_weight, params.ang_weight, params.ecal_weight, params.hist_weight]
dformat=params.dformat
thresh = params.thresh # threshold for data
angtype = params.angtype
particle = params.particle
lr = params.lr
energies = [110, 150, 190] # bins used in short analysis
if tlab:
    #datapath = '/gkhattak/data/*Measured3ThetaEscan/*.h5'
    weightdir = '/gkhattak/weights/3dgan_weights_' + params.name 
    pklfile = '/gkhattak/results/3dgan_history_' +  params.name +'.pkl'
    resultfile = '/gkhattak/results/3dgan_analysis_' +  params.name +'.pkl'
print(params)

Namespace(analyse=False, ang_weight=25, angtype='mtheta', ascale=1, aux_weight=0.1, batchsize=64, datapath='../../3dgan_files/data/LCD/EleMeasuredThetaEscan/*.h5', dformat='channels_last', ecal_weight=0.1, gen_weight=3, hist_weight=0.1, latentsize=256, lr=0.0008, nEvents=200000, name='training', nbepochs=60, particle='Ele', thresh=0.0, verbose=False, xpower=0.85, xscale=1, yscale=100)


# Build the networks

In [5]:
# Building discriminator and generator
gan.safe_mkdir(weightdir)
d=discriminator(xpower, dformat=dformat)
g=generator(latent_size, dformat=dformat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 51, 51, 25, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 51, 51, 25, 16)    2896      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 51, 51, 25, 16)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 51, 51, 25, 16)    0         
_________________________________________________________________
zero_padding3d_1 (ZeroPaddin (None, 51, 51, 27, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 47, 46, 22, 8)     23048     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 47, 46, 22, 8)     0         
__________

/Users/owoshch/Documents/CERN/binder/3Dgan/keras/AngleArch3dGAN.py:196: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  Model(input=[latent], output=[fake_image]).summary()
/Users/owoshch/Documents/CERN/binder/3Dgan/keras/AngleArch3dGAN.py:197: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  return Model(input=[latent], output=[fake_image])
